In [3]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Compose, Resize, ToTensor
from torch.optim import lr_scheduler

In [4]:
class Stanford40Dataset(Dataset):
    def __init__(self, img_dir, split_file, transform=None):
        self.img_dir = img_dir
        with open(split_file, 'r') as file:
            self.img_list = file.readlines()
        self.transform = transform
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_name = self.img_list[idx].rstrip('\n')
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')  # Ensure image is RGB
        label = img_name.split('_')[0]  # Extract action name from file name
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations
transform = Compose([
    Resize((256, 256)),
    ToTensor()
])

# Paths to the directories and split files
img_dir = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/JPEGImages'
train_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/train.txt'
test_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/test.txt'

# Create the dataset
train_dataset = Stanford40Dataset(img_dir=img_dir, split_file=train_split_file, transform=transform)
test_dataset = Stanford40Dataset(img_dir=img_dir, split_file=test_split_file, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
# Example to get one batch of data
for images, labels in train_loader:
    # Here images is a batch of images and labels is a batch of corresponding labels
    print(images.shape, labels)
    break  # Remove this break to iterate over the whole dataset

torch.Size([32, 3, 256, 256]) ('pouring', 'playing', 'rowing', 'watching', 'pushing', 'blowing', 'phoning', 'looking', 'playing', 'walking', 'smoking', 'playing', 'jumping', 'waving', 'pouring', 'using', 'pushing', 'playing', 'phoning', 'running', 'jumping', 'using', 'writing', 'writing', 'pushing', 'applauding', 'washing', 'playing', 'playing', 'throwing', 'fixing', 'riding')


In [7]:
# Define device for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
class EnhancedCNN(nn.Module):
    # Improved CNN architecture with more layers and advanced techniques
    def __init__(self, num_classes=40):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(128, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.global_avg_pool(x)
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate and print the model architecture
model = EnhancedCNN(num_classes=40).to(device)
print(model)


EnhancedCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=40, bias=True)
)


In [17]:
# Move the model to the chosen device
model.to(device)

# Assuming labels are strings, we need to convert them to integers for training
# Create a dictionary mapping each label to a unique integer
unique_labels = sorted(set([label for _, label in train_dataset]))
label_dict = {label: idx for idx, label in enumerate(unique_labels)}

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Optional: Define a learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Define the number of epochs for training
num_epochs = 100

# Function for validation phase
def validate(model, val_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), [label_dict[label] for label in labels]
            labels = torch.tensor(labels).to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()  # Set the model back to training mode
    return 100 * correct / total

best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Convert labels to tensor indices
        labels = torch.tensor([label_dict[label] for label in labels]).to(device)
        
        # Move the images to the device
        images = images.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    
    # Update the learning rate
    scheduler.step()
    
    # Print epoch loss
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')
    
    # Validation phase
    val_acc = validate(model, test_loader)
    print(f'Validation Accuracy: {val_acc}%')

    # Save the model if the accuracy is the best so far
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Model improved and saved with {best_acc}% accuracy')

print('Finished Training')

Epoch 1, Loss: 3.7034433517456056
Validation Accuracy: 1.7534345625451917%
Model improved and saved with 1.7534345625451917% accuracy
Epoch 2, Loss: 3.7012274532318115
Validation Accuracy: 1.7534345625451917%
Epoch 3, Loss: 3.7010088024139405
Validation Accuracy: 1.7534345625451917%
Epoch 4, Loss: 3.7009871082305907
Validation Accuracy: 1.7534345625451917%
Epoch 5, Loss: 3.700985462188721
Validation Accuracy: 1.7534345625451917%
Epoch 6, Loss: 3.700985439300537
Validation Accuracy: 1.7534345625451917%


KeyboardInterrupt: 

In [31]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.optim import lr_scheduler

class Stanford40Dataset(Dataset):
    def __init__(self, img_dir, split_file, transform=None, label_dict=None):
        self.img_dir = img_dir
        with open(split_file, 'r') as file:
            self.img_list = file.readlines()
        self.transform = transform
        self.label_dict = label_dict

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_name = self.img_list[idx].rstrip('\n')
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label_dict[img_name.split('_')[0]]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

transform = Compose([Resize((256, 256)), ToTensor(), Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
img_dir = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/JPEGImages'
train_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/train.txt'
test_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/test.txt'
unique_labels = sorted(set([line.split('_')[0] for line in open(train_split_file, 'r')]))
label_dict = {label: idx for idx, label in enumerate(unique_labels)}
train_dataset = Stanford40Dataset(img_dir, train_split_file, transform=transform, label_dict=label_dict)
test_dataset = Stanford40Dataset(img_dir, test_split_file, transform=transform, label_dict=label_dict)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class EnhancedCNN(nn.Module):
    def __init__(self, num_classes=40):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.75)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x)
        x = x.view(-1, 128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = EnhancedCNN(num_classes=40).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Place the validate function here
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 200
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()

    val_acc = validate(model, test_loader)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

print('Finished Training')


KeyboardInterrupt: 

In [30]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

# Call the test_model function
test_model(model, test_loader)


Accuracy of the model on the test images: 7.393347794649313%


In [43]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.optim import lr_scheduler

# Dataset class for Stanford 40
class Stanford40Dataset(Dataset):
    def __init__(self, img_dir, split_file, transform=None, label_dict=None):
        self.img_dir = img_dir
        with open(split_file, 'r') as file:
            self.img_list = file.readlines()
        self.transform = transform
        self.label_dict = label_dict

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_name = self.img_list[idx].rstrip('\n')
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label_dict[img_name.split('_')[0]]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

# Enhanced Transform with Augmentations
transform = Compose([
    Resize((256, 256)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    ToTensor(), 
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Paths and dataset loaders
img_dir = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/JPEGImages'
train_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/train.txt'
test_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/test.txt'
unique_labels = sorted(set([line.split('_')[0] for line in open(train_split_file, 'r')]))
label_dict = {label: idx for idx, label in enumerate(unique_labels)}

train_dataset = Stanford40Dataset(img_dir, train_split_file, transform=transform, label_dict=label_dict)
test_dataset = Stanford40Dataset(img_dir, test_split_file, transform=transform, label_dict=label_dict)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class EnhancedCNN(nn.Module):
    def __init__(self, num_classes=40, activation='tanh'):
        super(EnhancedCNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
        
        # Activation function selection
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()

    def forward(self, x):
        x = self.pool(self.activation(self.bn1(self.conv1(x))))
        x = self.pool(self.activation(self.bn2(self.conv2(x))))
        x = self.pool(self.activation(self.bn3(self.conv3(x))))
        x = self.global_pool(x)
        x = x.view(-1, 128)
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
# Validation function
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model and move it to the device
model = EnhancedCNN(num_classes=40, activation='tanh').to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer with the learning rate
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.05)

# Specify the number of epochs 
num_epochs = 5  

# Initialize the best validation accuracy variable
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    
    # Iterate over the training set
    for images, labels in train_loader:
        # Transfer images and labels to your device
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
    # Adjust the learning rate based on the scheduler
    scheduler.step()
    
    # Validation step after each epoch
    val_acc = validate(model, test_loader)
    print(f'Epoch {epoch+1}: Validation Accuracy: {val_acc}%')
    
    # Check if the validation accuracy is the best and save the model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

# Test function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Test the model after training
test_accuracy = test_model(model, test_loader)
print(f'Test Accuracy: {test_accuracy}%')

# Save test results to a file
with open('test_accuracy.txt', 'w') as file:
    file.write(f'Test Accuracy: {test_accuracy}%\n')

Epoch 1: Validation Accuracy: 11.171366594360087%
Epoch 2: Validation Accuracy: 11.478669558929862%
Epoch 3: Validation Accuracy: 12.79826464208243%
Epoch 4: Validation Accuracy: 13.322487346348518%
Epoch 5: Validation Accuracy: 13.376717281272596%
Test Accuracy: 13.250180766449747%


In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.optim import lr_scheduler
import copy

# Dataset class for Stanford 40
class Stanford40Dataset(Dataset):
    def __init__(self, img_dir, split_file, transform=None, label_dict=None):
        self.img_dir = img_dir
        with open(split_file, 'r') as file:
            self.img_list = file.readlines()
        self.transform = transform
        self.label_dict = label_dict

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_name = self.img_list[idx].rstrip('\n')
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label_dict[img_name.split('_')[0]]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

# Enhanced Transform with Augmentations
transform = Compose([
    Resize((256, 256)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    ToTensor(), 
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Paths and dataset loaders
img_dir = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/JPEGImages'
train_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/train.txt'
test_split_file = '/Users/mwr/Downloads/AVPR assignment 2/Stanford40/ImageSplits/test.txt'
unique_labels = sorted(set([line.split('_')[0] for line in open(train_split_file, 'r')]))
label_dict = {label: idx for idx, label in enumerate(unique_labels)}


train_dataset = Stanford40Dataset(img_dir, train_split_file, transform=transform, label_dict=label_dict)
test_dataset = Stanford40Dataset(img_dir, test_split_file, transform=transform, label_dict=label_dict)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

resnet_model = models.resnet18(pretrained=True)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 40)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(resnet_model.parameters(), lr=0.001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

def validate(model, val_loader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return val_loss / len(val_loader), 100 * correct / total


best_model_wts = copy.deepcopy(resnet_model.state_dict())
best_acc = 0.0
early_stopping_patience = 5
no_improvement_epochs = 0

num_epochs = 5  # Set  number of epochs

for epoch in range(num_epochs):
    resnet_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)

    val_loss, val_acc = validate(resnet_model, test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')
    scheduler.step(val_loss)

    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(resnet_model.state_dict())
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
        if no_improvement_epochs >= early_stopping_patience:
            print("Early stopping triggered")
            break

resnet_model.load_state_dict(best_model_wts)


# Test function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


test_accuracy = test_model(resnet_model, test_loader)
with open('resnet_test_results.txt', 'w') as file:
    file.write(f'Test Accuracy: {test_accuracy}%\n')


Epoch 1/5, Training Loss: 2.4899, Validation Loss: 2.3571, Validation Accuracy: 36.01%
Epoch 2/5, Training Loss: 1.8237, Validation Loss: 2.5072, Validation Accuracy: 37.85%
Epoch 3/5, Training Loss: 1.4577, Validation Loss: 2.0845, Validation Accuracy: 44.96%
Epoch 4/5, Training Loss: 1.2442, Validation Loss: 2.3500, Validation Accuracy: 41.09%
